Copyright &copy; 2017 D.S. Parker

# Final Exam Problem: Analyzing The Analyzers


The goal of this problem is to define "Data Scientists", based on the StackOverflow Tech Job Survey,
using nonnegative matrix factorization.

This is the method used in the O'Reilly booklet <b>Analyzing the Analyzers</b>
discussed at the start of this course.

For this problem,
however, you are asked to use the Stackoverflow data that you used in HW6,
rather than the survey the authors used.

(Stackoverflow does a large survey of tech workers on jobs:
https://insights.stackoverflow.com/survey )

In [80]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from patsy import dmatrices

import pandas as pd
from pandas import Series, DataFrame

import statsmodels.api as sm

from sklearn import datasets, svm

from sklearn.decomposition import NMF
from sklearn import preprocessing

## Part 0:  Getting the Survey data

First, go to the survey site and download the 2016 data:
https://insights.stackoverflow.com/survey
    
(The results of the 2017 survey are there, but the data is not yet available for download now.)

# IMPORTANT

For this problem, we want you to extract a subset of the data involving <i>data science</i>,
and compare the results for it with results for the entire dataset.

The <b>occupation</b> column in the dataset (i.e., column number 10)
involves job titles.  Please extract all rows that have
any of the job titles of interest here
(with the number of occurrences listed first):
<ul><li>
<b>Business intelligence or data warehousing expert</b> (392 occurrences)
</li><li>
<b>Data scientist</b> (800 occurrences)
</li><li>
<b>Developer with a statistics or mathematics background</b>  (960)
</li><li>
<b>Machine learning developer</b> (385).
</li></ul>

For this problem, extract all rows with these job titles into a <tt>DataScience</tt> table with about 2500 rows.

<!--
We also want you to consider the entire dataset; for simplicity let's call it <tt>JobSurvey</tt>.
-->

In [120]:
JobSurvey = pd.read_csv('2016 Stack Overflow Survey Results/2016 Stack Overflow Survey Responses.csv')

JobSurvey.shape

(56030, 66)

#  IN THIS ASSIGNMENT, DROP MISSING VALUES CAREFULLY

Instead of dropping all rows with nulls (a drastic solution), first selectively delete columns in a way that maximizes the size of the "data scientist" dataset.


## A nice way to avoid missing values:  some missing values are not 'NA's

Your alert classmate Phillip David discovered that the 'star_wars_vs_star_trek' column
was based on a question that permitted '' (empty string) as an answer.
This gets turned into NaN by Pandas when reading in data.

So this column should have something like '' instead of NaN
(after Pandas reads in the data).
Thus we can use .fillna('') to replace all the NaNs in this column.

In fact, Phillip showed that lots of columns can get fixed this way:
<pre>
self_identification,
education,
tech_do,
tech_want,
how_to_improve_interview_process,
why_stack_overflow,
dev_environment,
developer_challenges,
star_wars_vs_star_trek,
new_job_value
</pre>

The code below repairs each of these columns.

In [121]:
JobSurvey['star_wars_vs_star_trek'].head()

0          NaN
1    Star Wars
2          NaN
3          NaN
4          NaN
Name: star_wars_vs_star_trek, dtype: object

In [122]:
JobSurvey['star_wars_vs_star_trek'] = JobSurvey['star_wars_vs_star_trek'].fillna('')

JobSurvey['star_wars_vs_star_trek'].head()

0             
1    Star Wars
2             
3             
4             
Name: star_wars_vs_star_trek, dtype: object

In [123]:
Phillips_repairable_columns = 'self_identification,education,tech_do,tech_want,\
how_to_improve_interview_process,why_stack_overflow,dev_environment,\
developer_challenges,star_wars_vs_star_trek,new_job_value'.split(',')
#  the .split(',') method turns this into a list of strings

for column in Phillips_repairable_columns:
    JobSurvey[column] = JobSurvey[column].fillna('')
    
JobSurvey[Phillips_repairable_columns].head()

,self_identification,education,tech_do,tech_want,how_to_improve_interview_process,why_stack_overflow,dev_environment,developer_challenges,star_wars_vs_star_trek,new_job_value
0,Programmer,,,,,,,,,
1,Developer; Engineer; Programmer,I'm self-taught; On-the-job training; B.S. in ...,iOS; Objective-C,Swift,Show me more live code; Introduce me to the te...,To get help for my job; Because I can't do my ...,TextMate; Xcode,Unrealistic expectations; Interacting with non...,Star Wars,Salary; Ability to make or influence important...
2,,,,,,,,,,
3,Engineer,I'm self-taught; On-the-job training,,,Be more flexible about interview scheduling; O...,To get help for my job; Beacause I love to learn,Visual Studio,Corporate policies; Interacting with non-techn...,,Office location; Remote working option; Flexib...
4,Developer; Engineer; Programmer; Sr. Developer...,,,,,,,,,


##  Another column that can be fixed:  The Big Mac Index

The 'Big Mac Index' is the Jan 2016 local price of a Big Mac divided by dollar price,
and is inferrable from Country.  See:  http://www.economist.com/content/big-mac-index

The index is only tracked by the Economist in 56 countries, and so is not always entered.
Since it is undefined mainly by poor countries, we just fillna(...) with the highest value.

In [124]:
max_big_mac_index = JobSurvey['big_mac_index'].max()
max_big_mac_index

6.4400000000000004

In [125]:
JobSurvey['big_mac_index'] = JobSurvey['big_mac_index'].fillna(max_big_mac_index)

In [126]:
JobSurvey['big_mac_index'].head()

0    6.44
1    6.44
2    6.44
3    6.44
4    6.44
Name: big_mac_index, dtype: float64

## Taking Stock:  Find out how many values are non-null in each column

In [127]:
JobSurvey.count()

Unnamed: 0                          56030
collector                           56030
country                             55528
un_subregion                        55313
so_region                           55390
age_range                           55727
age_midpoint                        55336
gender                              55586
self_identification                 56030
occupation                          49519
occupation_group                    46934
experience_range                    49520
experience_midpoint                 49520
salary_range                        46121
salary_midpoint                     41742
big_mac_index                       56030
tech_do                             56030
tech_want                           56030
aliens                              49177
programming_ability                 46982
employment_status                   49576
industry                            40110
company_size_range                  39932
team_size_range                   

## Taking Stock for "data scientists": determining which columns have the most missing values

With this information, we can omit columns that have the most missing values (below).

In [128]:
JobSurvey[ JobSurvey.occupation == 'Data scientist' ].count()

Unnamed: 0                          800
collector                           800
country                             796
un_subregion                        795
so_region                           795
age_range                           796
age_midpoint                        786
gender                              797
self_identification                 800
occupation                          800
occupation_group                    800
experience_range                    795
experience_midpoint                 795
salary_range                        735
salary_midpoint                     686
big_mac_index                       800
tech_do                             800
tech_want                           800
aliens                              791
programming_ability                 746
employment_status                   797
industry                            677
company_size_range                  666
team_size_range                     666
women_on_team                       665


In [129]:
JobSurvey[ JobSurvey.occupation == 'Data scientist' ][0:4]

,Unnamed: 0,collector,country,un_subregion,so_region,age_range,age_midpoint,gender,self_identification,occupation,...,important_sameend,important_newtech,important_buildnew,important_buildexisting,important_promotion,important_companymission,important_wfh,important_ownoffice,developer_challenges,why_stack_overflow
28,34171,Facebook,Andorra,Southern Europe,Western Europe,20-24,22.0,Female,Hacker,Data scientist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
34,29315,Facebook,Argentina,South America,South America,20-24,22.0,Male,Developer; Engineer; Sr. Developer; Full-stack...,Data scientist,...,This is somewhat important,This is somewhat important,This is very important,I don't care about this,This is very important,This is somewhat important,This is somewhat important,I don't care about this,Fragile code base; Poor documentation; Changin...,To get help for my job; To give help to others
205,31023,Facebook,Australia,Australia and New Zealand,Oceania,35-39,37.0,Female,Programmer,Data scientist,...,I don't care about this,This is very important,This is somewhat important,This is somewhat important,This is very important,This is very important,This is somewhat important,This is somewhat important,Poor team dynamics; Inefficient development pr...,To get help for my job
225,49442,Facebook,Australia,Australia and New Zealand,Oceania,20-24,22.0,Male,Developer; Engineer; Programmer,Data scientist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,


In [130]:
JobSurvey.columns[0]  # strangely named because the column name was omitted

'Unnamed: 0'

In [131]:
JobSurvey.shape

(56030, 66)

In [132]:
JobSurvey[['jobId']] = JobSurvey[['Unnamed: 0']]
JobSurvey.shape

(56030, 67)

In [133]:
DataScienceJobSurvey = JobSurvey[ JobSurvey.occupation == 'Data scientist' ]
DataScienceJobSurvey[0:5]

,Unnamed: 0,collector,country,un_subregion,so_region,age_range,age_midpoint,gender,self_identification,occupation,...,important_newtech,important_buildnew,important_buildexisting,important_promotion,important_companymission,important_wfh,important_ownoffice,developer_challenges,why_stack_overflow,jobId
28,34171,Facebook,Andorra,Southern Europe,Western Europe,20-24,22.0,Female,Hacker,Data scientist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,34171
34,29315,Facebook,Argentina,South America,South America,20-24,22.0,Male,Developer; Engineer; Sr. Developer; Full-stack...,Data scientist,...,This is somewhat important,This is very important,I don't care about this,This is very important,This is somewhat important,This is somewhat important,I don't care about this,Fragile code base; Poor documentation; Changin...,To get help for my job; To give help to others,29315
205,31023,Facebook,Australia,Australia and New Zealand,Oceania,35-39,37.0,Female,Programmer,Data scientist,...,This is very important,This is somewhat important,This is somewhat important,This is very important,This is very important,This is somewhat important,This is somewhat important,Poor team dynamics; Inefficient development pr...,To get help for my job,31023
225,49442,Facebook,Australia,Australia and New Zealand,Oceania,20-24,22.0,Male,Developer; Engineer; Programmer,Data scientist,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,49442
241,2229,Facebook,Austria,Western Europe,Western Europe,50-59,54.5,Male,Engineer; Programmer; Sr. Developer; Expert,Data scientist,...,This is very important,This is very important,This is very important,I don't care about this,This is somewhat important,This is very important,This is somewhat important,Unrealistic expectations; Poor documentation; ...,To get help for my job; Beacause I love to learn,2229


# Problem:  Factor the Survey Matrix for Data Scientists using Nonnegative Matrix factorization

This is asking you to produce the matrix factorization of data scientists that you can.
The factors obtained will correspond to "classes" of data scientist.


To do this, first extract all rows with these job titles
into a <tt>DataScience</tt> table with about 2500 rows.

After eliminating as few columns and rows as possible,
perform nonnegative matrix factorization.

Finally, report the clearest factors you obtain this way.
These factors should identify types of "data scientist",
just as described in the Appendix of <b>Analyzing the Analyzers</b>.

# Part 0:  Extract all "data scientists", using the 4 types of job titles

As mentioned above, you are supposed to include all 4 kinds of job titles in this model:
<ul><li>
<b>Business intelligence or data warehousing expert</b> (392 occurrences)
</li><li>
<b>Data scientist</b> (800 occurrences)
</li><li>
<b>Developer with a statistics or mathematics background</b>  (960)
</li><li>
<b>Machine learning developer</b> (385).
</li></ul>

In [134]:
#This is similar to hw6.
DataScience = JobSurvey.copy()
df1 = DataScience[ DataScience.occupation == 'Business intelligence or data warehousing expert' ]
df2 = DataScience[ DataScience.occupation == 'Data scientist' ]
df3 = DataScience[ DataScience.occupation == 'Developer with a statistics or mathematics background' ]
df4 = DataScience[ DataScience.occupation == 'Machine learning developer' ]
frames = [df1, df2, df3, df4]
DataScience = pd.concat(frames)

In [135]:
DataScience.head()

,Unnamed: 0,collector,country,un_subregion,so_region,age_range,age_midpoint,gender,self_identification,occupation,...,important_newtech,important_buildnew,important_buildexisting,important_promotion,important_companymission,important_wfh,important_ownoffice,developer_challenges,why_stack_overflow,jobId
245,3080,Facebook,Austria,Western Europe,Western Europe,25-29,27.0,Female,Engineer,Business intelligence or data warehousing expert,...,This is very important,This is very important,This is very important,This is somewhat important,This is very important,This is somewhat important,This is somewhat important,Unrealistic expectations; Poor documentation; ...,To get help for my job; To receive help on my ...,3080
359,52419,Facebook,Austria,Western Europe,Western Europe,30-34,32.0,Male,Developer; Expert; Full-stack Developer,Business intelligence or data warehousing expert,...,This is somewhat important,This is somewhat important,This is somewhat important,I don't care about this,This is somewhat important,This is somewhat important,I don't care about this,Inefficient development processes; Unrealistic...,To get help for my job; To communicate with ot...,52419
662,46022,Facebook,Brazil,South America,South America,50-59,54.5,Male,Developer; Engineer; Programmer,Business intelligence or data warehousing expert,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,46022
779,51412,Facebook,Brazil,South America,South America,40-49,44.5,Male,Manager,Business intelligence or data warehousing expert,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,51412
1213,46457,Facebook,Denmark,Northern Europe,Western Europe,40-49,44.5,Male,Developer; Sr. Developer; Expert; Full-stack D...,Business intelligence or data warehousing expert,...,This is very important,This is somewhat important,This is somewhat important,This is somewhat important,This is somewhat important,This is somewhat important,This is somewhat important,Inefficient development processes; Poor docume...,To communicate with others like me,46457


# Part 1: drop columns that have the most missing values for data scientists

In [136]:
#  Drop (or repair) columns from JobSurvey that have the most missing values.
#   The goal is to get a good matrix factorization.
Phillips_repairable_columns = 'self_identification,education,tech_do,tech_want,\
how_to_improve_interview_process,why_stack_overflow,dev_environment,\
developer_challenges,star_wars_vs_star_trek,new_job_value'.split(',')
#  the .split(',') method turns this into a list of strings

for column in Phillips_repairable_columns:
    DataScience[column] = JobSurvey[column].fillna('')
    
DataScience[Phillips_repairable_columns].head()

#Fill Big_mac_index with highest value
max_big_mac_index = DataScience['big_mac_index'].max()
DataScience['big_mac_index'] = DataScience['big_mac_index'].fillna(max_big_mac_index)    


In [137]:
###
### You can implement the method devised by your alert classmate Anna Merkoulovitch --
### She dropped all columns in which more than 10% of the values were missing.
###
### She told us that this got rid of some columns that didn't seem likely to help --
### including questions about aliens, cats and dogs -- and saved a lot of data.
###
threshold = DataScience.shape[0] * 0.9        
DataScience = DataScience.dropna(thresh=int(threshold), axis=1)

#DataScience = DataScience.drop('jobId', axis=1)
DataScience = DataScience.drop('Unnamed: 0', axis=1)

In [138]:
 DataScience.isnull().sum()

collector                             0
country                              15
un_subregion                         20
so_region                            18
age_range                            11
age_midpoint                         36
gender                               18
self_identification                   0
occupation                            0
occupation_group                      0
experience_range                     16
experience_midpoint                  16
salary_range                        189
big_mac_index                         0
tech_do                               0
tech_want                             0
aliens                               40
programming_ability                 155
employment_status                    10
dev_environment                       0
commit_frequency                    159
hobby                               158
dogs_vs_cats                        247
desktop_os                          167
unit_testing                        160


In [139]:
#  Determine how many "data scientist" rows are left over after your careful dropping of columns.
DataScience[ DataScience.occupation == 'Data scientist' ].shape


(800, 35)

In [140]:
DataScience.shape

(2537, 35)

# Part 2: drop all rows that still have null values (with .dropna)

In [141]:
DataScience = DataScience.dropna()

DataScience.shape

(1943, 35)

In [142]:
 DataScience.isnull().sum()

collector                           0
country                             0
un_subregion                        0
so_region                           0
age_range                           0
age_midpoint                        0
gender                              0
self_identification                 0
occupation                          0
occupation_group                    0
experience_range                    0
experience_midpoint                 0
salary_range                        0
big_mac_index                       0
tech_do                             0
tech_want                           0
aliens                              0
programming_ability                 0
employment_status                   0
dev_environment                     0
commit_frequency                    0
hobby                               0
dogs_vs_cats                        0
desktop_os                          0
unit_testing                        0
rep_range                           0
visit_freque

#  Part 3:  do the Nonnegative Matrix Factorization

In [143]:
#  consider for example:
#  http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html

In [144]:
#  NMF factors a (n x p) dataset matrix X  into   X = W H
#   where W is (n x k) and H is (k x p), and k is the desired number of factors for NMF.
#   Thus the H matrix is a matrix of "factors" involving the p variables in X.
#   The factors are "weight vectors" that are interpreted like principal components.

In [145]:
le = preprocessing.LabelEncoder()
DataScience_trans = DataScience.apply(le.fit_transform)

In [146]:
nwf = NMF(n_components=4, random_state=1234, init='random')
DataScience_fit = nwf.fit_transform(DataScience_trans)
DataScience_comps = nwf.components_

In [147]:
DataScience_comps_df = pd.DataFrame(data=DataScience_comps, columns=DataScience.columns.values)

In [153]:
DataScience_comps_df= DataScience_comps_df.drop('jobId', axis=1) #don't need jobId

In [157]:
DataScience_comps_df.head()

,collector,country,un_subregion,so_region,age_range,age_midpoint,gender,self_identification,occupation,occupation_group,...,unit_testing,rep_range,visit_frequency,why_learn_new_tech,education,new_job_value,how_to_improve_interview_process,star_wars_vs_star_trek,developer_challenges,why_stack_overflow
0,0.042083,1.265698,0.213982,0.216892,0.036203,0.060655,0.024612,2.691210,0.047410,0.022204,...,0.062702,0.101563,0.043546,0.060811,2.879122,75.905445,3.832668,0.047740,0.000000,2.908847
1,0.029453,1.310314,0.248183,0.212512,0.037749,0.061765,0.019124,3.495378,0.023447,0.018013,...,0.038625,0.110128,0.050304,0.062829,2.669384,0.000000,1.566862,0.023083,0.000000,1.057928
2,0.225299,2.891698,0.336740,0.352712,0.111112,0.129766,0.051376,6.236897,0.080550,0.038874,...,0.125539,0.162075,0.076803,0.085897,5.023018,0.000000,5.784187,0.057233,0.000000,3.579555
3,0.011798,1.235523,0.278434,0.214270,0.055318,0.078940,0.022873,3.320660,0.035590,0.019444,...,0.043202,0.109133,0.041195,0.062793,3.798718,0.950010,2.998137,0.038018,43.468525,2.976172


In [264]:
df_1 = DataScience_comps_df.apply(np.sort, axis=1)
#df_2 = DataScience_comps_df.loc[[1]].sort(ascending=False, axis=1)
#df_3 = DataScience_comps_df.loc[[2]].sort(ascending=False, axis=1)
#df_4 = DataScience_comps_df.loc[[3]].sort(ascending=False, axis=1)



In [257]:
df_1.head() #scroll to right

,collector,country,un_subregion,so_region,age_range,age_midpoint,gender,self_identification,occupation,occupation_group,...,unit_testing,rep_range,visit_frequency,why_learn_new_tech,education,new_job_value,how_to_improve_interview_process,star_wars_vs_star_trek,developer_challenges,why_stack_overflow
0,0.0,0.000000,0.003620,0.021725,0.022204,0.024612,0.030040,0.036203,0.042083,0.043095,...,0.275700,0.472970,1.161062,1.265698,2.691210,2.879122,2.908847,3.832668,8.884166,75.905445
1,0.0,0.000000,0.015598,0.018013,0.018708,0.019124,0.020883,0.023083,0.023447,0.029453,...,0.250321,0.472425,1.057928,1.310314,1.566862,2.669384,3.495378,8.102616,36.729382,54.968952
2,0.0,0.000000,0.000000,0.000000,0.030595,0.038874,0.051376,0.051650,0.057233,0.071562,...,0.336740,0.352712,0.644108,1.116683,2.891698,3.579555,5.023018,5.784187,6.236897,16.156799
3,0.0,0.011798,0.019095,0.019444,0.020277,0.022873,0.026069,0.035590,0.038018,0.041195,...,0.424487,0.950010,1.235523,2.976172,2.998137,3.320660,3.798718,10.884988,23.061733,43.468525


# Top 3 components are:

## why_stack_overflow, developer_challenges, starwars_vs_star_trek